In [1]:
# Importation des librairies
from my_function import ( 
    sel_classif_pixel,
    report_from_dict_to_df,
    supprimer_dossier_non_vide,
    rasterization,
    id_construction,
    stratified_grouped_validation,
    save_classif)
import os
import matplotlib.pyplot as plt
from osgeo import gdal
import geopandas as gpd
import numpy as np
import pandas as pd
import sys
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import (StratifiedKFold, StratifiedGroupKFold)
from sklearn.metrics import (confusion_matrix, classification_report,
    accuracy_score)
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw
import classification as cla
import plots

In [2]:
# Création du dossier de sauvegarde temporaire et paramètres
racine = "/home/onyxia/work"
output_dir = os.path.join(racine,"output_classif")
os.makedirs(output_dir, exist_ok=True) # Création d'un dossier output temporaire

path_sample = os.path.join(racine,"Depot_Git/results/data/sample/Sample_BD_foret_T31TCJ.shp")
path_sample_px = os.path.join(output_dir,"sample_classif_px.shp")

sample_rasterized = os.path.join(output_dir,"rasterized_sample.tif")
path_image_3b = os.path.join(racine,"Depot_Git/results/data/img_pretraitees/Serie_temp_S2_3_bands.tif")
path_image_10b = os.path.join(racine,"Depot_Git/results/data/img_pretraitees/Serie_temp_S2_10_bands.tif")
path_image_allbands = os.path.join(racine,"Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif")

path_sample_px_centroid = os.path.join(output_dir,"sample_px_centroid.shp")
path_sample_px_id = os.path.join(output_dir,"sample_px_id.shp")
path_rasterized_sample_id = os.path.join(output_dir, "rasterized_sample_id.tif")

suffix = '_CV{}folds_stratified_group_x{}times'.format(nb_folds, nb_iter)
out_figs_dir = os.path.join(racine,"Depot_Git/results/figure")
out_matrix = os.path.join(out_figs_dir, 'matrice{}.png'.format(suffix))
out_qualite = os.path.join(out_figs_dir, 'qualites{}.png'.format(suffix))

out_classif = os.path.join(racine, "Depot_Git/results/data/classif/carte_essences_echelle_pixel.tif")

In [3]:
# Save d'un vecteur echantillons avec que les données pour classif pixel
sample = gpd.read_file(path_sample)
sample_px = sel_classif_pixel(sample[['Code',"geometry"]])
sample_px.to_file(path_sample_px)

In [ ]:
# Rasterisation des échantillons pour la classification pixel
rasterization(
    in_vector=path_sample_px,
    out_image=sample_rasterized,
    field_name='Code',
    ref_image=path_image_allbands,
    dtype='Byte'
)

In [ ]:
# Méthode stratifiée simple
# 1 --- define parameters
# inputs
sample_filename = sample_rasterized
image_filename = path_image_3b

# Sample parameters
#test_size = 0.7
nb_iter = 30
nb_folds = 5
is_point = False
# if is_point is True
field_name = 'Code'

# outputs
suffix = '_CV{}fold_stratifed_x{}times'.format(nb_folds, nb_iter)
out_folder = output_dir
out_classif = os.path.join(out_folder, 'ma_classif{}.tif'.format(suffix))
out_matrix = os.path.join(out_folder, 'ma_matrice{}.png'.format(suffix))
out_qualite = os.path.join(out_folder, 'mes_qualites{}.png'.format(suffix))

print("Extraction des échantillons")
# 2 --- extract samples
if not is_point :
    X, Y, t = cla.get_samples_from_roi(image_filename, sample_filename)
else :
    # get X
    list_row, list_col = rw.get_row_col_from_file(sample_filename, image_filename)
    image = rw.load_img_as_array(image_filename)
    X = image[(list_row, list_col)]

    # get Y
    gdf = gpd.read_file(sample_filename)
    Y = gdf.loc[:, field_name].values
    Y = np.atleast_2d(Y).T

list_cm = []
list_accuracy = []
list_report = []

print("KFold début")
kf = StratifiedKFold(n_splits=nb_folds)
for train, test in kf.split(X, Y):
    print(train,test)
    X_train, X_test = X[train], X[test]
    Y_train, Y_test = Y[train], Y[test]

    print("Entrainement")
    # 3 --- Train
    rfc = RandomForestClassifier(
        max_depth = 50,
        oob_score = True,
        max_samples = 0.75,
        class_weight = 'balanced',
        n_jobs = -1
    )
    rfc.fit(X_train, Y_train[:,0])

    print("Predict")
    # 4 --- Test
    Y_predict = rfc.predict(X_test)

    # compute quality
    list_cm.append(confusion_matrix(Y_test, Y_predict))
    list_accuracy.append(accuracy_score(Y_test, Y_predict))
    report = classification_report(Y_test, Y_predict,

                                   labels=np.unique(Y_predict), output_dict=True)

    # store them
    list_report.append(report_from_dict_to_df(report))

# compute mean of cm
array_cm = np.array(list_cm)
mean_cm = array_cm.mean(axis=0)

# compute mean and std of overall accuracy
array_accuracy = np.array(list_accuracy)
mean_accuracy = array_accuracy.mean()
std_accuracy = array_accuracy.std()

# compute mean and std of classification report
array_report = np.array(list_report)
mean_report = array_report.mean(axis=0)
std_report = array_report.std(axis=0)
a_report = list_report[0]
mean_df_report = pd.DataFrame(mean_report, index=a_report.index,
                              columns=a_report.columns)
std_df_report = pd.DataFrame(std_report, index=a_report.index,
                             columns=a_report.columns)

# Display confusion matrix
plots.plot_cm(mean_cm, np.unique(Y_predict))
plt.savefig(out_matrix, bbox_inches='tight')

# Display class metrics
fig, ax = plt.subplots(figsize=(10, 7))
ax = mean_df_report.T.plot.bar(ax=ax, yerr=std_df_report.T, zorder=2)
ax.set_ylim(0.5, 1)
_ = ax.text(1.5, 0.95, 'OA : {:.2f} +- {:.2f}'.format(mean_accuracy,
                                                      std_accuracy),
            fontsize=14)
ax.set_title('Class quality estimation')

# custom : cuteness
# background color
ax.set_facecolor('ivory')
# labels
x_label = ax.get_xlabel()
ax.set_xlabel(x_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
y_label = ax.get_ylabel()
ax.set_ylabel(y_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
# borders
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.tick_params(axis='x', colors='darkslategrey', labelsize=14)
ax.tick_params(axis='y', colors='darkslategrey', labelsize=14)
# grid
ax.minorticks_on()
ax.yaxis.grid(which='major', color='darkgoldenrod', linestyle='--',
              linewidth=0.5, zorder=1)
ax.yaxis.grid(which='minor', color='darkgoldenrod', linestyle='-.',
              linewidth=0.3, zorder=1)
plt.savefig(out_qualite, bbox_inches='tight')

In [ ]:
# Construction du raster id pour la méthode stratifiée groupée
id_construction(sample_px,path_sample_px_id)
rasterization(
    in_vector=path_sample_px_id,
    out_image=path_rasterized_sample_id,
    field_name="id",
    ref_image=path_image_allbands
)

In [ ]:
id_filename = path_rasterized_sample_id
nb_iter = 1
nb_folds = 2
rfc, list_cm, list_accuracy, list_report, Y_predict =\
stratified_grouped_validation(
    nb_iter=nb_iter,
    nb_folds=nb_folds,
    sample_filename=sample_rasterized,
    image_filename=path_image_3b,
    id_filename=id_filename
)

In [ ]:
# Entrainement stratifié groupé
# 1 --- define parameters
# inputs
sample_filename = sample_rasterized
image_filename = path_image_allbands
id_filename = os.path.join(output_dir, "rasterized_sample_id.tif")

# Sample parameters
#test_size = 0.7
nb_iter = 30
nb_folds = 5
is_point = False
# if is_point is True
field_name = 'Code'

# outputs
suffix = '_CV{}folds_stratified_group_x{}times'.format(nb_folds, nb_iter)
out_folder = "/home/onyxia/work"
out_classif = os.path.join(out_folder, 'ma_classif{}.tif'.format(suffix))
out_matrix = os.path.join(out_folder, 'ma_matrice{}.png'.format(suffix))
out_qualite = os.path.join(out_folder, 'mes_qualites{}.png'.format(suffix))


# 2 --- extract samples
if not is_point :
    X, Y, t = cla.get_samples_from_roi(image_filename, sample_filename)
    _, groups, _ = cla.get_samples_from_roi(image_filename, id_filename)
else :
    # get X
    list_row, list_col = rw.get_row_col_from_file(sample_filename, image_filename)
    image = rw.load_img_as_array(image_filename)
    X = image[(list_row, list_col)]

    # get Y
    gdf = gpd.read_file(sample_filename)
    Y = gdf.loc[:, field_name].values
    Y = np.atleast_2d(Y).T

list_cm = []
list_accuracy = []
list_report = []
groups = np.squeeze(groups)

# Iter on stratified K fold
for i in range(nb_iter):
    print (f"Début de la {i+1} itération")
    kf = StratifiedGroupKFold(n_splits=nb_folds, shuffle=True)
    for train, test in kf.split(X, Y, groups=groups):
        X_train, X_test = X[train], X[test]
        Y_train, Y_test = Y[train], Y[test]

        # 3 --- Train
        rfc = RandomForestClassifier(
            max_depth = 50,
            oob_score = True,
            max_samples = 0.75,
            class_weight = 'balanced',
            n_jobs = -1
        )
        rfc.fit(X_train, Y_train[:,0])

        # 4 --- Test
        Y_predict = rfc.predict(X_test)

        # compute quality
        list_cm.append(confusion_matrix(Y_test, Y_predict))
        list_accuracy.append(accuracy_score(Y_test, Y_predict))
        report = classification_report(Y_test, Y_predict,
                                        labels=np.unique(Y_predict),
                                        output_dict=True,
                                        zero_division = 0)

        # store them
        list_report.append(report_from_dict_to_df(report))

In [ ]:
# Sauvegarder les listes dans des fichier JSON

import json
list_json = [list_cm[i].tolist() for i in range (len(list_cm))]
with open(os.path.join(racine,"list_cm.json"), "w", encoding="utf-8") as fichier:
    json.dump(list_json, fichier, indent=4, ensure_ascii=False)
with open(os.path.join(racine,"list_accuracy.json"), "w", encoding="utf-8") as fichier:
    json.dump(list_accuracy, fichier, indent=4, ensure_ascii=False)
list_json = [df.to_dict(orient="records") for df in list_report]
with open(os.path.join(racine,"list_report.json"), "w", encoding="utf-8") as fichier:
    json.dump(list_json, fichier, indent=4, ensure_ascii=False)

In [7]:
# Chargement des listes

import json

with open(os.path.join(racine,"list_report.json"), "r", encoding="utf-8") as fichier:
    list_json = json.load(fichier)
    fichier = None
list_report = [pd.DataFrame(i) for i in list_json]

with open(os.path.join(racine,"list_cm.json"), "r", encoding="utf-8") as fichier:
    list_json = json.load(fichier)
    fichier = None
list_cm = [np.array(list_json[i]) for i in range(len(list_json))]

with open (os.path.join(racine,"list_accuracy.json"), "r", encoding="utf-8") as fichier :
    list_json = json.load(fichier)
list_accuracy = list_json.copy()

In [ ]:
# Vérification des cm

a = 0
for cm in list_cm:
    if len(cm) == 9:
        a += 1
print (f"{a}/{len(list_cm)} matrices de confusions qui sont correctes")

In [ ]:
# Vérification des rapports

a = 0
for report in list_report:
    nb_report = len(report.keys())
    if nb_report == 9:
        a += 1
print (f"{a}/{len(list_report)} rapports qui sont corrects")

In [ ]:
# Stratégie d'évitement : supprimer les tableaux qui n'ont pas toutes les classes
list_report_2 = []
list_cm_2 = []
for report in list_report:
    nb_report = len(report.keys())
    if nb_report == 9 :
        list_report_2.append(report)
print(len(list_report_2))
for cm in list_cm:
    nb_cm = len(cm)
    if nb_cm == 9:
        list_cm_2.append(cm)
print(len(list_cm_2))


In [ ]:
## Plots
# compute mean of cm
array_cm = np.array(list_cm_2)
mean_cm = array_cm.mean(axis=0)

# compute mean and std of overall accuracy
array_accuracy = np.array(list_accuracy)
mean_accuracy = array_accuracy.mean()
std_accuracy = array_accuracy.std()

# compute mean and std of classification report
array_report = np.array(list_report_2)
mean_report = array_report.mean(axis=0)
std_report = array_report.std(axis=0)
a_report = list_report_2[0]
mean_df_report = pd.DataFrame(mean_report, index=a_report.index,
                              columns=a_report.columns)
std_df_report = pd.DataFrame(std_report, index=a_report.index,
                             columns=a_report.columns)

# Display confusion matrix
plots.plot_cm(mean_cm, np.unique(Y_predict))
plt.savefig(out_matrix, bbox_inches='tight')

# Display class metrics
fig, ax = plt.subplots(figsize=(10, 7))
ax = mean_df_report.T.plot.bar(ax=ax, yerr=std_df_report.T, zorder=2)
ax.set_ylim(0.5, 1)
_ = ax.text(1.5, 0.95, 'OA : {:.2f} +- {:.2f}'.format(mean_accuracy,
                                                      std_accuracy),
            fontsize=14)
ax.set_title('Class quality estimation')

# custom : cuteness
# background color
ax.set_facecolor('ivory')
# labels
x_label = ax.get_xlabel()
ax.set_xlabel(x_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
y_label = ax.get_ylabel()
ax.set_ylabel(y_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
# borders
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.tick_params(axis='x', colors='darkslategrey', labelsize=14)
ax.tick_params(axis='y', colors='darkslategrey', labelsize=14)
# grid
ax.minorticks_on()
ax.yaxis.grid(which='major', color='darkgoldenrod', linestyle='--',
              linewidth=0.5, zorder=1)
ax.yaxis.grid(which='minor', color='darkgoldenrod', linestyle='-.',
              linewidth=0.3, zorder=1)
plt.savefig(out_qualite, bbox_inches='tight')

In [12]:
save_classif(
    image_filename=path_image_3b,
    model=rfc,
    out_classif=out_classif
)

In [12]:
# Ecriture de la classif (obsolète)

X_img, _, t_img = cla.get_samples_from_roi(path_image_allbands, path_image_allbands)
Y_predict = rfc.predict(X_img)
image_filename = path_image_allbands

# Get image dimension
ds = rw.open_image(image_filename)
nb_row, nb_col, _ = rw.get_image_dimension(ds)

#initialization of the array
img = np.zeros((nb_row, nb_col, 1), dtype='uint8')
#np.Y_predict

img[t_img[0], t_img[1], 0] = Y_predict
image_filename = path_image_allbands

out_filename = '/home/onyxia/work/Depot_Git/results/data/classif/carte_essences_echelle_pixel.tif'

# get reference dataset
ds = rw.open_image(image_filename)

# just replace the number of band (1 instead of 3)
rw.write_image(out_filename, img, data_set=ds, gdal_dtype=gdal.GDT_Byte,
                transform=None, projection=None, driver_name=None,
                nb_col=None, nb_ligne=None, nb_band=1)

In [13]:
## Nettoyage des dossiers
supprimer_dossier_non_vide(output_dir)